<a href="https://colab.research.google.com/github/mbednarski/sda-rnn/blob/master/Forecasting_przygotowanie_danych.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importy
Zaczymamy od zaimportowania potrzebnych bibliotek

In [ ]:
import os
import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf

## Wczytanie danych
Na początek pobobieramy i rozpakowujemy dane.

In [ ]:
zip_path = tf.keras.utils.get_file(
    origin='https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip',
    fname='jena_climate_2009_2016.csv.zip',
    extract=True)
csv_path, _ = os.path.splitext(zip_path)


Pobrany przez nas plik zawiera pomiary różnych parametrów pogodowych. Spórzmy na kilka pierwszysch wpisów aby wyrobić sobie intuicję.

In [ ]:
df = pd.read_csv(csv_path)
df

Widzimy, że dane zawierają wiele parmetrów. Naszym celem będzie predykcja temperatury (T (degC)). W tym celu będziemy używać nie tylko temperatury historycznej, ale także innych zmierzonych wartości.

Dane były zbieranie z okresem 10 minut. Możemy powiedzieć że częstotliwość próbkowanania wynosi 5 razy na godzinę. W tym przypadku jest to ważne, gdyż temperatura ma charakter ciągły i pomiar odbywa się co pewien czas. W przypadku danych dyskretnych (np. sekwencja DNA, tekst) - nie mamy do czynienia z czasem pobrania próbki - po prostu mamy kolejne, następujące po sobie wartości.

Na nasze potrzeby będziemy potrzebowali danych próbkowanych co godzinę. Dlatego najpierw wybierzmy odpowiednie wiersze:


In [ ]:
df = df[5::6].copy()
df

## EDA i preprocessing
Przyjrzyjmy sie rozkładowi danych:

In [ ]:
df.describe()

###Pytanie 1 (łatwe)
Czy widzisz jakiś problem?

### Pytanie 2 (średnie)
Czy wiesz dlaczego wiatr jest podany w dwóch kolumnach? (wd (deg) oraz wv (m/s)). Pytanie może być bardzo trudnie jeżeli nigdy nie miałeś_aś styczności z takimi danymi.

###Pytanie 3 (trudne)
W jaki inny sposób możemy opisać dane nt. siły i kierunku wiatru? Jaki jest problem z obecną reprezentacją?

## Poprawianie outlierów. 

Wartość -9999 jest najprawdopodbniej błędna. Zamieńmy ją na zero.

### Pytanie 4 (średnie)
Dlaczego nie chcemy po prostu usunąć tych rekordów?

In [ ]:
bad_wind = df['wv (m/s)'] < 0 
bad_max_wind =  df['max. wv (m/s)'] < 0
print('Ilość rekordów z wadliwym wiatrem', bad_wind.sum())
print('Ilość rekordów z wadliwym porywem wiatru', bad_max_wind.sum())
df.loc[bad_wind, 'wv (m/s)'] = 0
df.loc[bad_max_wind, 'max. wv (m/s)'] = 0


Wiemy już, że azymut nie jest dobrą cechą. Najprościej będzie się jej pozbyć. Przy okazji usuńmy kolumnę ze stemplem czasowym. Nie będziemy go używać w modelu.

In [ ]:
df = df.drop(columns=['wd (deg)', 'Date Time'])
df

### Zadanie 1 (trudne)
Zaimplementuj kod który z kolumn wv (m/s) i wd (deg) utworzy dwie nowe - wind_x i wind_y. Nowe kolumny będą przechowywały wektor opisujący siłę i kierunek wiatru. Potrzebna będzie trygonometria ;)

## Fearture engineering

### Pytanie 5 (trudne)
Pogoda zmienia się sezonowo (w zimie i w nocy jest raczej zimniej niż latem i w dzień). W jaki sposób możemy wykonać feature engineering który doda odpowiednie kolumny?

### Zadanie 2 (trudne)
Zaimplementuj odpowiedź na pytanie 5 ;)

### Zadanie 3 (bardzo trudne)
Założyliśmy w ciemno, że temperatra zmienia się w cyklu dziennym i rocznym. Sprawdź czym jest szybka transformata fouriera i wykorzystaj ją do potwierdzenia/obalenia naszej intuicji. UWAGA: zadanie wymaga dużej pracy własnej i przyswojenia konkretnej dawki wiedzy.

# Podział na zbiory treningowy/walidacyjny i testowy

Podzielmy dane na zbiory train/val/test. Zwykle podziału dokonujemy losowo, tym razem jest inaczej. Pierwsze 70% trafi do zbiory treningowego, kolejne 15% do walidacyjnego a ostatne 15% do testowego. Zapewni to bardziej wiarygodną ewaluację - model testujemy na danych późniejszych niż treningowe. Dodatkowo w ten sposób będziemy mieli zapewnioną spójność wygenerowanych okien - o co chodzi dowiesz się wkrótce ;)

In [ ]:
dset_len = len(df)
df_train = df.iloc[0:int(dset_len*0.7)]
df_val = df.iloc[int(dset_len*0.7):int(dset_len*0.85)]
df_test = df.iloc[int(dset_len*0.85):]

num_features = df.shape[1]
len(df_train), len(df_val), len(df_test)

## Normalizacja/standaryzacja

Sieć neuronowa potrzebuje danych numerycznych w standardowej postaci. Tzn, średnia każdej cechy powinna wynosić zero a jej odchylenie standardowe jeden. Dobra wiadomość jest taka, że dowolne dane możemy przekształcić do takiej formy przy użyciu wzoru:
$$ z= \frac{x-μ}{σ}$$

UWAGA, średnią i odchylenie obliczamy ze zbioru treningowego! To bardzo ważne, inaczej mamy wyciek ze zbiorów val/test.

### Pytanie 6 (trudne)
Dlaczego jest to ważne?

In [ ]:
train_mean = df_train.mean()
train_std = df_train.std()

df_train = (df_train - train_mean) / train_std
df_val = (df_val - train_mean) / train_std
df_test = (df_test - train_mean) / train_std

In [ ]:
df_train.describe().round(3)

Przyjrzyjmy sie jak rozład wygląda teraz:

(W razie potrzemy omówimy funkcję `melt`)

In [ ]:
ax = sns.violinplot(x='variable', y='value', data=df_train.melt())
ax.grid()
_ = ax.set_xticklabels(df_train.keys(), rotation=90)

To samo dla zbioru walidacyjnego

In [ ]:
ax = sns.violinplot(x='variable', y='value', data=df_val.melt())
ax.grid()
_ = ax.set_xticklabels(df_val.keys(), rotation=90)

## Zapisanie zbiorów

Mamy gotowe dane, teraz wystarczy je zapisać.

In [ ]:
df_train.to_csv('temp_train.csv')
df_val.to_csv('temp_val.csv')
df_test.to_csv('temp_test.csv')


# Przygotowamie danych do użycia w modelu

Nadal nie wiemy jak zakodować dane tak aby użyć ich w modelu. To za chwilę się zmiemi ;) 

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df=df_train, val_df=df_val, test_df=df_test,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

In [ ]:
def plot(self, model=None, plot_col='T (degC)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue


    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      if len(predictions.shape) == 2:
        predictions = tf.expand_dims(predictions,1)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

In [ ]:
def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [ ]:
next_in_12_h = WindowGenerator(input_width=12, label_width=1, shift=1, label_columns=['T (degC)'])
print(next_in_12_h)
next_in_12_h.plot()

In [ ]:
lookahead_24h = WindowGenerator(input_width=24, label_width=1, shift=24, label_columns=['T (degC)'])
print(lookahead_24h)
lookahead_24h.plot()

In [ ]:
continous_24h = WindowGenerator(input_width=48, label_width=24, shift=24, label_columns=['T (degC)'])
print(continous_24h)
continous_24h.plot()

## Pytanie 7 (średnie/trudne)

Jaki baseline możemy wymyślić?

### Pytanie 8 (trudne)

Czy istnieje górna granica skuteczności modelu której nie możemy przekroczyć?